In [32]:
import pandas as pd
import numpy as np
import json
from math import log, floor
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./Data/preprocessed_data.csv')
with open('./Data/feature_config.json') as json_data:
  features = json.load(json_data)

features.keys()

len_columns = features['length']+ features['core_count']+ features['unique']+\
  features['blosum_sum']+ features['blosum']
len_fea = df[len_columns]
len_fea = len_fea.divide(len_fea.max())
df.drop(columns=len_columns, inplace = True)
df = df.join(len_fea)

origin_cols = df.columns
onehot_columns = features['hla']+features['core']+features['PFR']+features['cluster']
one_hot = pd.get_dummies(data=df, columns=onehot_columns)
one_hot.drop(columns = set(origin_cols)-set(onehot_columns), inplace = True)


In [55]:
zxc = df.copy().dropna(axis = 1)

In [58]:
cols_to_drop = ['peptide','core','LPFR','RPFR','true_index','n_label']
data = zxc.drop(columns = cols_to_drop, axis = 1)
y = data['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
data.drop(columns = 'aff', inplace = True)
data.fillna(0,inplace = True)
xtrain, xvalid, ytrain, yvalid = train_test_split(data, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)
result = {}
# Scale the data to (0,1) for Bayesyes
scl = MinMaxScaler()
scl.fit(xtrain)
xtrain_svd_scl = scl.transform(xtrain)
xvalid_svd_scl = scl.transform(xvalid)

In [3]:
def hla_preprocess(df):
  hla_encoder = LabelEncoder()
  #hla_encoder.fit(df['hla'])
  #np.save('Data/hla_encoder_classes.npy', hla_encoder.classes_)
  hla_encoder.classes_ = np.load('./Data/hla_encoder_classes.npy')
  df['hla'] = hla_encoder.transform(df['hla'])


  return df

v = pd.read_csv('./Data/vectors_data.csv')
v = hla_preprocess(v)
df = pd.merge(df, v, on = ['peptide', 'hla'], how = 'left')

columns = features['blosum']+features['unique']+features['core_tfidf']+[c for c in list(v.columns) if c not in ['peptide', 'hla']]+['aff']
cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

In [41]:
data = one_hot.join(df[columns]).drop_duplicates()
y = data['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
data.drop(columns = 'aff', inplace = True)
data.fillna(0,inplace = True)
xtrain, xvalid, ytrain, yvalid = train_test_split(data, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)
result = {}
# Scale the data to (0,1) for Bayesyes
scl = MinMaxScaler()
scl.fit(xtrain)
xtrain_svd_scl = scl.transform(xtrain)
xvalid_svd_scl = scl.transform(xvalid)

In [59]:
#Logistic Regression
LR = LogisticRegression(C=1.0)
LR.fit(xtrain_svd_scl, ytrain)
predictions = LR.predict_proba(xvalid_svd_scl)[:,1]
result['LR'] = roc_auc_score(yvalid,predictions)
result['LR']

0.7520200975711848

In [43]:
X = one_hot.join(df[columns]).drop_duplicates()#.reset_index(drop = True)
X = X.dropna().reset_index(drop = True)
y = X['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
X.drop(columns = 'aff', inplace = True)
#X.fillna(0,inplace = True)

oof_preds = np.zeros(X.shape[0])

for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y):
  pred = {}

  xtrain, xvalid = X.loc[train_index], X.loc[test_index]
  ytrain, yvalid = y.loc[train_index], y.loc[test_index]

  scl = MinMaxScaler()
  scl.fit(xtrain)
  xtrain_svd_scl = scl.transform(xtrain)
  xvalid_svd_scl = scl.transform(xvalid)
  
  rf = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs = -1)
  rf.fit(xtrain_svd_scl, ytrain)
  predictions = rf.predict_proba(xvalid_svd_scl)[:,1]
  
  oof_preds[test_index] = predictions
  
  print(roc_auc_score(yvalid,predictions))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.9min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.3s finished


0.999991856400442


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.2min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.3s finished


0.9999925058136676


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.5s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.4s finished


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.4s finished


0.9999903263238078


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.5s


0.9999911170249711


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.5s finished


In [42]:
rf = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs = -1)
rf.fit(xtrain_svd_scl, ytrain)
predictions = rf.predict_proba(xvalid_svd_scl)[:,1]
result['RF'] = roc_auc_score(yvalid,predictions)
result['RF']

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.5s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.4s finished


0.7368923008017985

In [8]:
#rf = RandomForestClassifier(max_depth=3, n_estimators=500)
rf_enc = OneHotEncoder(categories='auto')
rf_lm = LogisticRegression(C=1.0)
rf_lm.fit(rf_enc.fit_transform(rf.apply(xtrain_svd_scl)), ytrain)

predictions = rf_lm.predict_proba(rf_enc.transform(rf.apply(xvalid_svd_scl)))[:,1]
result['RF_LR'] = roc_auc_score(yvalid,predictions)
result['RF_LR']

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.4s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.2s finished


0.9835669217990557

In [10]:
gbes = GradientBoostingClassifier(
  n_estimators=100,
  validation_fraction=0.2,
  n_iter_no_change=5, tol=0.01,
  random_state=0
)
gbes.fit(xtrain_svd_scl, ytrain)
predictions = gbes.predict_proba(xvalid_svd_scl)[:,1]
result['GB'] = roc_auc_score(yvalid,predictions)
result['GB']

0.7321672065028977

In [12]:
grd_enc = OneHotEncoder(categories='auto')
gb_lr = LogisticRegression(C=1.0)
gb_lr.fit(grd_enc.fit_transform(gbes.apply(xtrain_svd_scl)[:, :, 0]), ytrain)

y_pred_grd_lm = gb_lr.predict_proba(
  grd_enc.transform(grd.apply(xvalid_svd_scl)[:, :, 0]))[:, 1]
result['gb_lr'] = roc_auc_score(yvalid,y_pred_grd_lm)
result['gb_lr']

NameError: name 'grd' is not defined